In [67]:
import pandas as pd
import gzip

# Cambia la ruta al archivo que descargues
file = "flightlist_20200401_20200430.csv.gz"

with gzip.open(file, 'rt') as f:
    df = pd.read_csv(f)



In [ ]:
# Guardo el dataset de los continentes
url_continents = "https://datahub.io/core/airport-codes/r/airport-codes.csv"
df_continents = pd.read_csv(url_continents)


# Elimino las columnas que no son necesarias
df_continents.drop(["ident", "name", "elevation_ft", "iso_region", "municipality", "gps_code", "local_code"], axis=1, inplace=True)

# Parece ser que donde pone NaN en verdad es NA
df_continents["continent"] = df_continents["continent"].fillna("NA")

# Elimino las filas que no tienen codigo ICAO
df_continents = df_continents[df_continents["icao_code"].notna()]

# Elimino las filas que no tienen codigo IATA
df_continents = df_continents[df_continents["iata_code"].notna()]

# Elimino aquellos que no sean aeropuertos
df_continents = df_continents[~df_continents["type"].isin(["heliport", "seaplane_base"])]

# Tambien voy a cambiar la forma en que se muestra la latitud y longitud
df_continents["latitud"] = df_continents["coordinates"].apply(lambda x: float(x.split(",")[0]))
df_continents["longitud"] = df_continents["coordinates"].apply(lambda x: float(x.split(",")[1]))

# Elimino la columna de coordenadas
df_continents.drop(["coordinates"], axis=1, inplace=True)

# Para renombrar los paises usare un dataframe de soporte
url_iso = "https://datahub.io/core/country-list/r/data.csv"
df_iso = pd.read_csv(url_iso)  

df_continents = df_continents.merge(df_iso, left_on='iso_country', right_on='Code', how='left')
df_continents = df_continents.rename(columns={'Name': 'country_name'}).drop(columns=['Code'])

# Tambien voy a renombrar los continentes para que sean mas amigables
continent_names = {
    'AF': 'Africa',
    'AN': 'Antarctica',
    'AS': 'Asia',
    'EU': 'Europe',
    'NA': 'North America',
    'OC': 'Oceania',
    'SA': 'South America'
}
df_continents['continent'] = df_continents['continent'].replace(continent_names)


In [69]:
df_continents["continent"].unique()

array(['Oceania', 'North America', 'Europe', 'Asia', 'Africa',
       'South America', 'Antarctica'], dtype=object)

In [78]:
df_continents

,type,continent,iso_country,icao_code,iata_code,coordinates,country_name,latitud,longitud
0,small_airport,Oceania,SB,AGAF,AFT,"-9.191389, 160.948611",Solomon Islands,160.948611,-9.191389
1,small_airport,Oceania,SB,AGAR,RNA,"-9.86054358262, 161.979546547",Solomon Islands,161.979547,-9.860544
2,small_airport,Oceania,SB,AGAT,ATD,"-8.87333, 161.011002",Solomon Islands,161.011002,-8.873330
3,small_airport,Oceania,SB,AGBA,VEV,"-7.912779808044434, 156.70599365234375",Solomon Islands,156.705994,-7.912780
4,small_airport,Oceania,SB,AGBT,BPF,"-8.56202777778, 158.119305556",Solomon Islands,158.119306,-8.562028
...,...,...,...,...,...,...,...,...,...
7506,medium_airport,Asia,CN,ZYTN,TNH,"42.2538888889, 125.703333333",China,125.703333,42.253889
7507,large_airport,Asia,CN,ZYTX,SHE,"41.639801, 123.483002",China,123.483002,41.639801
7508,medium_airport,Asia,CN,ZYXC,XEN,"40.580328, 120.700374",China,120.700374,40.580328
7509,medium_airport,Asia,CN,ZYYJ,YNJ,"42.8828010559, 129.451004028",China,129.451004,42.882801


In [76]:
df_continents[['longitud', 'latitud']] = df_continents['coordinates'].apply(pd.Series)


ValueError: Columns must be same length as key

In [71]:
df

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,HBAL102,NaN,a21c0f,NaN,NaN,NaN,NaN,2020-03-31 00:41:44+00:00,2020-04-01 22:50:47+00:00,2020-04-01 00:00:00+00:00,-4.821136,-76.319370,18288.0,-4.524902,-79.010274,2194.56
1,AUA1028,NaN,440081,OE-LPD,B772,YSSY,LOWW,2020-03-31 03:29:15+00:00,2020-04-01 00:41:43+00:00,2020-04-01 00:00:00+00:00,-33.963489,151.180630,0.0,48.117417,16.549988,NaN
2,ABW120,NaN,424564,VQ-BRJ,B748,KORD,UUEE,2020-03-31 07:07:17+00:00,2020-04-01 12:54:29+00:00,2020-04-01 00:00:00+00:00,41.965911,-87.883152,0.0,55.983490,37.495839,472.44
3,CSN461,CZ461,780da8,B-2026,B77L,KLAX,EDDF,2020-03-31 07:15:52+00:00,2020-04-01 12:02:53+00:00,2020-04-01 00:00:00+00:00,33.936302,-118.413963,0.0,50.035675,8.572299,45.72
4,ETH3730,NaN,04015c,NaN,NaN,OMSJ,EBLG,2020-03-31 08:05:10+00:00,2020-04-01 01:15:08+00:00,2020-04-01 00:00:00+00:00,25.386425,55.422139,609.6,50.634247,5.439027,76.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842900,SKQ33,NaN,a8f75b,N677ST,PC12,1NC8,KBUY,2020-04-30 23:46:16+00:00,2020-04-30 23:59:42+00:00,2020-04-30 00:00:00+00:00,35.545147,-80.236193,1524.0,36.035431,-79.459419,563.88
842901,N9989A,NaN,adf3ef,NaN,NaN,62GA,62GA,2020-04-30 23:46:41+00:00,2020-04-30 23:56:52+00:00,2020-04-30 00:00:00+00:00,33.379025,-83.897431,304.8,33.365158,-83.842987,640.08
842902,N853BP,NaN,abb2a4,N853BP,AS50,KEBG,KMFE,2020-04-30 23:46:46+00:00,2020-04-30 23:57:59+00:00,2020-04-30 00:00:00+00:00,26.423423,-98.138439,304.8,26.180191,-98.236481,30.48
842903,N5810F,NaN,a77bfd,N5810F,PRM1,KSNA,KF70,2020-04-30 23:46:48+00:00,2020-04-30 23:57:22+00:00,2020-04-30 00:00:00+00:00,33.667370,-117.874374,0.0,33.602057,-117.106934,609.60


In [ ]:
# Como sabemos que es de un mes y solo son dias, voy a quitar toda la informacion irrelevante
df["day"] = df["day"].apply(lambda x: x.split()[0].split("-")[2])

# Da distintos valores de latitud y longitud para los mismos aeropuertos, quitare todas estas posiciones
# y usare las posiciones que hay en df_continentes

# Además eliminare los elemntos que no sean necesarios

df.drop(["callsign", "number", "icao", "registration", "typecode", "firstseen", "lastseen",
         "latitude_1", "longitude_1, altitude_1", "latitude_2", "longitude_2, altitude_2"], axis=1, inplace=True)